# Tutorial 1: Framing the Question

**Week 2, Day 1: Modeling Practice**

**By Neuromatch Academy**


__Content creators:__ Marius 't Hart, Megan Peters, Paul Schrater, Gunnar Blohm, Konrad Kording, Marius Pachitariu

__Content reviewers:__ Eric DeWitt, Tara van Viegen, Marius Pachitariu

__Production editors:__ Ella Batty, Spiros Chavlis

# Reflection

- Learned about the 10 steps to modelling, with a major focus on steps 1-4, which are:
  * phenomenon/question/goal.
  * conducting a literature review.
  * finding model ingredients.
  * formulating a mathematically defined hypothesis.

- Two types of models:
  * Computational model = try to develop an explanation or implement hypothesis. Helps with mechanistic explanations.
  * Data-driven model = tests the computational model and raises new questions.

---
# Tutorial objectives
Last week you gained some understanding of what models can buy us in neuroscience. But how do you build a model? Today, we will try to clarify the process of computational modeling, by thinking through the logic of modeling based on your project ideas. We will now work through the 4 first steps of modeling ([Blohm et al., 2019](https://doi.org/10.1523/ENEURO.0352-19.2019)):

**Framing the question**

1. finding a phenomenon and a question to ask about it
2. understanding the state of the art
3. determining the basic ingredients
4. formulating specific, mathematically defined hypotheses

### Demos
We will demo the modeling process to you based on the train illusion. In parallel to the computational model, you will develop questions and hypotheses for a data analysis project based on the same phenomenon. This will be done during “Think!” sections.

Enjoy!


In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# for random distributions:
from scipy.stats import norm, poisson

# for logistic regression:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
# @title Figure settings
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

In [3]:
# @title Plotting Functions

def rasterplot(spikes,movement,trial):

  [movements, trials, neurons, timepoints] = np.shape(spikes)

  trial_spikes = spikes[movement,trial, :, :]

  trial_events = [((trial_spikes[x,:] > 0).nonzero()[0]-150)/100 for x in range(neurons)]

  plt.figure()
  dt=1/100
  plt.eventplot(trial_events, linewidths=1);
  plt.title('movement: %d - trial: %d'%(movement, trial))
  plt.ylabel('neuron')
  plt.xlabel('time [s]')
  plt.show()

def plotCrossValAccuracies(accuracies):
  f, ax = plt.subplots(figsize=(8, 3))
  ax.boxplot(accuracies, vert=False, widths=.7)
  ax.scatter(accuracies, np.ones(8))
  ax.set(xlabel="Accuracy", yticks=[],
         title=f"Average test accuracy: {accuracies.mean():.2%}")
  ax.spines["left"].set_visible(False)
  plt.show()

In [4]:
# @title Generate Data

def generateSpikeTrains():

  gain = 2
  neurons = 50
  movements = [0, 1, 2]
  repetitions = 800

  np.random.seed(37)

  # set up the basic parameters:
  dt = 1/100
  start, stop = -1.5, 1.5
  t = np.arange(start, stop+dt, dt)  # a time interval
  Velocity_sigma = 0.5  # std dev of the velocity profile
  Velocity_Profile = norm.pdf(t, 0, Velocity_sigma)/norm.pdf(0, 0, Velocity_sigma)  # The Gaussian velocity profile, normalized to a peak of 1

  # set up the neuron properties:
  Gains = np.random.rand(neurons) * gain  # random sensitivity between 0 and `gain`
  FRs = (np.random.rand(neurons) * 60 ) - 10  # random base firing rate between -10 and 50

  # output matrix will have this shape:
  target_shape = [len(movements), repetitions, neurons, len(Velocity_Profile)]

  # build matrix for spikes, first, they depend on the velocity profile:
  Spikes = np.repeat(Velocity_Profile.reshape([1, 1, 1, len(Velocity_Profile)]),
                     len(movements)*repetitions*neurons,axis=2).reshape(target_shape)

  # multiplied by gains:
  S_gains = np.repeat(np.repeat(Gains.reshape([1, 1, neurons]), len(movements)*repetitions, axis=1).reshape(target_shape[:3]),
                      len(Velocity_Profile)).reshape(target_shape)
  Spikes = Spikes * S_gains

  # and multiplied by the movement:
  S_moves = np.repeat( np.array(movements).reshape([len(movements), 1, 1, 1]),
                      repetitions*neurons*len(Velocity_Profile), axis=3 ).reshape(target_shape)
  Spikes = Spikes * S_moves

  # on top of a baseline firing rate:
  S_FR = np.repeat(np.repeat(FRs.reshape([1, 1, neurons]),
                             len(movements)*repetitions, axis=1).reshape(target_shape[:3]),
                   len(Velocity_Profile)).reshape(target_shape)
  Spikes = Spikes + S_FR

  # can not run the poisson random number generator on input lower than 0:
  Spikes = np.where(Spikes < 0, 0, Spikes)

  # so far, these were expected firing rates per second, correct for dt:
  Spikes = poisson.rvs(Spikes * dt)

  return(Spikes)


def subsetPerception(spikes):

  movements = [0, 1, 2]
  split = 400
  subset = 40
  hwin = 3

  [num_movements, repetitions, neurons, timepoints] = np.shape(spikes)

  decision = np.zeros([num_movements, repetitions])

  # ground truth for logistic regression:
  y_train = np.repeat([0, 1, 1],split)
  y_test = np.repeat([0, 1, 1],repetitions-split)

  m_train = np.repeat(movements, split)
  m_test = np.repeat(movements, split)

  # reproduce the time points:
  dt = 1/100
  start, stop = -1.5, 1.5
  t = np.arange(start, stop+dt, dt)

  w_idx = list( (abs(t) < (hwin*dt)).nonzero()[0] )
  w_0 = min(w_idx)
  w_1 = max(w_idx)+1  # python...

  # get the total spike counts from stationary and movement trials:
  spikes_stat = np.sum( spikes[0, :, :, :], axis=2)
  spikes_move = np.sum( spikes[1:, :, :, :], axis=3)

  train_spikes_stat = spikes_stat[:split, :]
  train_spikes_move = spikes_move[:, :split, :].reshape([-1, neurons])

  test_spikes_stat = spikes_stat[split:, :]
  test_spikes_move = spikes_move[:, split:, :].reshape([-1, neurons])

  # data to use to predict y:
  x_train = np.concatenate((train_spikes_stat, train_spikes_move))
  x_test  = np.concatenate((test_spikes_stat, test_spikes_move))

  # this line creates a logistics regression model object, and immediately fits it:
  population_model = LogisticRegression(solver='liblinear', random_state=0).fit(x_train, y_train)

  # solver, one of: 'liblinear', 'newton-cg', 'lbfgs', 'sag', and 'saga'
  # some of those require certain other options
  #print(population_model.coef_)       # slope
  #print(population_model.intercept_)  # intercept

  ground_truth = np.array(population_model.predict(x_test))
  ground_truth = ground_truth.reshape([3, -1])

  output = {}
  output['perception'] = ground_truth
  output['spikes'] = spikes[:, split:, :subset, :]

  return(output)


def getData():

  spikes = generateSpikeTrains()

  dataset = subsetPerception(spikes=spikes)

  return(dataset)


dataset = getData()
perception = dataset['perception']
spikes = dataset['spikes']

----
# Step 1: Finding a phenomenon and a question to ask about it


Video 1: Asking a question.

Pitfalls:
* Vague question
* Question is a toolkit or model-first
* Precise aspect of phenomenon to model is unclear [solution: literature review?]
* No clear goal behind the question
*

As a reminder, here is what you should discuss and write down:

* What exact aspect of this neural with *N* neurons and *M* trials data needs modeling?
* Define an evaluation method!
  * How will you know your modeling is good?
  * E.g., comparison to specific data (quantitative method of comparison?)
* Think of an experiment that could test your model.

**Note**

The hardest part is Step 1. Once that is properly set up, all the others should be easier. **BUT**: often you think that Step 1 is done only to figure out in later steps (anywhere really) that you were not as clear on your question and goal than you thought. Revisiting Step 1 is a frequent necessity. Don't feel bad about it. You can revisit Step 1 later; for now, let's move onto the next step.

----
# Step 2: Understanding the state of the art & background


Here you will learn how to do a literature review (**to be done for your project AFTER this tutorial!**).

----
# Step 3: Determining the basic ingredients


Pitfalls:

* I’m experienced, I don’t need to think about ingredients anymore

* I can’t think of any ingredients (stimuli, parameters, controls, measurements?)

* I can’t think of any links (= mechanisms) between the inputs and outputs.

----
# Step 4: Formulating specific, mathematically defined hypotheses



* I don’t need hypotheses, I will just play around with the model
* My hypotheses don’t match my question (or vice versa)
* I can’t write down a math hypothesis (... means you lack ingredients *or* you have a structural hypothesis - expecting a certain model component to be crucial in explaining the phenomenon)


If you want to learn more about steps 5-10 of modelling from ([Blohm et al., 2019](https://doi.org/10.1523/ENEURO.0352-19.2019)), you can later read the paper, or check out the optional notebook available [here](https://compneuro.neuromatch.io/projects/modelingsteps/ModelingSteps_5through10.html). However, at the Neuromatch Academy, we would like you to use a new app built specifically to help with projects, the NMA project planner.

----
# NMA project planner

For the rest of the project time today and until the end of the summer school, you will use an online tool ([here](https://nma-project-planner.vercel.app)) that will help you organize the questions, hypotheses, datasets, methods and other ingredients for the projects. The tool is built on a large language model that takes your answers and compares them to the requirements of the section, then gives you feedback about how well the answers match the requirements and what you can do to improve your score. The fields you complete are persistent on the computer where you complete them, and they can be easily saved/loaded and shared between the group. Start filling out this tool together (one person shares their screen), and then share the saved file with everyone in the group.

Note that for NMA we do not consider it important what order you complete the project planner in. For some projects, it might make sense to start with the dataset, since you need to understand this well before you can formulate questions about it. For other types of projects, you might want to start with the question, for example if you plan to make your own model of a phenomenon, like in the train illusion. The only thing that matters is that at the end of the summer school, all sections have been filled, and you have good scores on all sections (7 or above, we won’t check, don’t worry).

Watch Konrad tell you more about [the app](https://nma-project-planner.vercel.app) he has built in the video below!

----
# Summary

In this tutorial, we worked through some steps of the process of modeling.

- We defined a phenomenon and formulated a question (step 1)
- We collected information the state-of-the-art about the topic (step 2)
- We determined the basic ingredients (step 3), and used these to formulate a specific mathematically defined hypothesis (step 4)

We further introduced the NMA project planner, which will guide you throughout the entire project development over the next two weeks. For the rest of today, you will use the project planner and the additional guidance in the project guide to make progress. Any progress is good progress, on any of the sections in the project planner. Different projects may progress through the sections in a different order, but what matters is that at the end all the sections are filled out. Find the order that works best for you and your team, and revisit each section as often as needed to make a solid overall project plan.

----
# Reading
Blohm G, Kording KP, Schrater PR (2020). A How-to-Model Guide for Neuroscience. _eNeuro_, 7(1) ENEURO.0352-19.2019. doi: [10.1523/ENEURO.0352-19.2019](https://doi.org/10.1523/ENEURO.0352-19.2019)

Kording KP, Blohm G, Schrater P, Kay K (2020). Appreciating the variety of goals in computational neuroscience. _Neurons, Behavior, Data Analysis, and Theory_ 3(6). arXiv: [arxiv:2002.03211v1](https://arxiv.org/abs/2002.03211v1), url: [https://nbdt.scholasticahq.com/article/16723-appreciating-the-variety-of-goals-in-computational-neuroscience](https://nbdt.scholasticahq.com/article/16723-appreciating-the-variety-of-goals-in-computational-neuroscience)

Schrater PR, Peters MK, Kording KP, Blohm G (2019). Modeling in Neuroscience as a Decision Process. _OSF pre-print_. url: [https://osf.io/w56vt/](https://osf.io/w56vt/)